# Introduction to Dask graphs

In this notebook we see how to run functions concurrently using Dask graphs.

We are going to use a simple example: computing the square of two numbers and adding the results. A real use case would be, of course, when dealing with time-consuming functions. That could be expensive IO operations, for instance. We are going to simulate expensive IO operation by adding a `time.sleep(1)` to our functions.

On a regular python program (unless we use numpy arrays) we would do the following:
 - x1 = square of the first number
 - x2 = square of the second number
 - x = x1 + x2
 
Using Dask, we are going to transform that to
 - (x1 = square of the first number, x2 = square of the first number)  # computed at the same time!
 - x = x1 + x2

In [ ]:
import dask
import time

In [ ]:
def square(n):
    time.sleep(2)
    return n * n
    
def add(m, n):
    time.sleep(2)
    return m + n

In [ ]:
%%time 

x = square(1)
y = square(2)
z = add(x, y)

***

## Building a computational graph

In [ ]:
x = dask.delayed(square)(1)
y = dask.delayed(square)(2)
z = dask.delayed(add)(x, y)

In [ ]:
z.visualize(rankdir='LR')

In [ ]:
z.visualize(rankdir='LR', optimize_graph=True, color='order',
            cmap='autumn', node_attr={'penwidth': '2'})

In [ ]:
%%time
z.compute()

***

## Questions

<mark>**Question 1**</mark>: Rewrite the following cell so it's executed lazily.
 * Which functions should be `delayed`? `square`? `sum`? Both? Why?
 * Visualize the graph.
 * Compare the execution time with the sequential execution.

In [ ]:
x = [square(i) for i in range(10)]
y = sum(x)

In [ ]:
%load solutions/exercise1.py

<mark>**Question 2**</mark>: Try a number of `square` calls larger and then shorter than the number of threads of the processor (24 for the gpu partition of Piz Daint). How much time it's going to take for 24 calls and how much for 25?

***
<mark>**Question 3**</mark>: Rewrite the following cell so it's executed lazily.
 * Which functions should be delayed?
 * Visualize the graph.
 * Compare the execution time with the sequential execution.

In [ ]:
x = []
for i in range(10):
    x.append(square(i))

y = sum(x)

In [ ]:
%load solutions/exercise2.py

***
<mark>**Question 4**</mark>: Rewrite the following cell so it's executed lazily.
 * Which functions should be delayed?
 * Visualize the graph.

In [ ]:
x = []
for i in range(10):
    if i % 2 == 0:
        x.append(square(i))
    else:
        x.append(add(i, i))

y = sum(x)

In [ ]:
%load solutions/exercise3.py